In [34]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import creds

# Set your credentials here
CLIENT_ID = creds.CLIENT_ID  # Replace with your Client ID
CLIENT_SECRET = creds.CLIENT_SECRET  # Replace with your Client Secret
REDIRECT_URI = 'http://localhost:8888/callback'  # Redirect URI
SCOPE = "playlist-modify-public playlist-modify-private"

# Authenticate and get access token
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                               client_secret=CLIENT_SECRET,
                                               redirect_uri=REDIRECT_URI,
                                               scope=SCOPE))

# Debugging output to verify authentication
print("Successfully authenticated with Spotify!")


Successfully authenticated with Spotify!


In [35]:
# Function to fetch tracks from a specific playlist
def get_playlist_tracks(playlist_id):
    tracks = []
    try:
        results = sp.playlist_tracks(playlist_id)
        
        # Check if 'items' exist in the results and if it's not empty
        if 'items' not in results or not results['items']:
            print("No tracks found in the playlist.")
            return []
        
        # Extract track names, artists, and Spotify URLs from the first batch
        for item in results['items']:
            track = item['track']
            tracks.append({
                'name': track['name'],
                'artist': ', '.join([artist['name'] for artist in track['artists']]),
                'url': track['external_urls']['spotify']
            })
        
        # Check if there are more pages of tracks and get them
        while results['next']:
            results = sp.next(results)
            for item in results['items']:
                track = item['track']
                tracks.append({
                    'name': track['name'],
                    'artist': ', '.join([artist['name'] for artist in track['artists']]),
                    'url': track['external_urls']['spotify']
                })
        
        return tracks
    except Exception as e:
        print(f"An error occurred while fetching playlist tracks: {e}")
        return []

# Replace with the specific playlist ID you want to fetch tracks from
playlist_id = '1yYjTgTdhYLhnzFMcUqsVb'  # Example: '37i9dQZF1DXcBWIGoYBM5M'

# Fetch and display tracks from the specified playlist
tracks = get_playlist_tracks(playlist_id)

if tracks:
    print(f"Tracks from Playlist ID: {playlist_id}")
    for track in tracks:
        print(f"  Track Name: {track['name']}")
else:
    print(f"No tracks found in playlist {playlist_id}")

Tracks from Playlist ID: 1yYjTgTdhYLhnzFMcUqsVb
  Track Name: Gulabi Aankhen
  Track Name: Kaun Tujhe (From "M.S.Dhoni - The Untold Story")
  Track Name: Lambiyaan Si Judaiyaan (From "Raabta")
  Track Name: Suno- Na Sangemarmar (From "Youngistaan)
  Track Name: Milne Hai Mujhse Aayi (From "Aashiqui 2")
  Track Name: Senorita
  Track Name: Raabta
  Track Name: Bulleya
  Track Name: Mere Rashke Qamar (From "Baadshaho")
  Track Name: Bhar Do Jholi Meri
  Track Name: Kinna Sona (From "Marjaavaan")
  Track Name: BIBA
  Track Name: Sunn Raha Hai (Male Version)
  Track Name: Kabhi Jo Baadal Barse (From "Jackpot)
  Track Name: Wajah Tum Ho
  Track Name: Badtameez Dil
  Track Name: Kabira
  Track Name: Ae Dil Hai Mushkil Title Track
  Track Name: Hui Malang (From "Malang - Unleash The Madness")
  Track Name: Sawan Mein Lag Gayi Aag
  Track Name: Afghan Jalebi (Film Version)
  Track Name: Dus Bahane 2.0 (From "Baaghi 3") (feat. K.K., Shaan, Tulsi Kumar)
  Track Name: Main Badhiya Tu Bhi Badhiya 

In [42]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler


class RecommendationModel:
    def __init__(self, sp):
        # Load the dataset
        self.spotify_df = pd.read_csv('SpotifySongs.csv')

        # Check for required columns
        if 'ArtistName' not in self.spotify_df.columns or 'SongName' not in self.spotify_df.columns:
            raise KeyError("Dataset must contain 'SongName' and 'ArtistName' columns")

        # Create a combined column for song and artist names
        self.spotify_df['song_artist'] = self.spotify_df['SongName'] + " " + self.spotify_df['ArtistName']
        self.spotify_df.drop_duplicates('song_artist', inplace=True, ignore_index=True)

        # Create the feature set
        self.complete_feature_set = self.create_feature_set(self.spotify_df)
        self.sp = sp
        print("Spotify API client successfully initialized.")

    def create_feature_set(self, df):
        # TF-IDF vectorization
        tfidf = TfidfVectorizer(stop_words='english')
        tfidf_matrix = tfidf.fit_transform(df['song_artist'])
        tfidf_df = pd.DataFrame(
            tfidf_matrix.toarray(),
            columns=['tfidf_' + str(i) for i in tfidf.get_feature_names_out()],
            index=df.index
        )

        # Normalize numerical columns
        numerical_cols = ['Popularity', 'Danceability', 'Energy', 'Key', 'Loudness', 'Mode', 'Speechiness',
                          'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo', 'Duration_ms']
        numerical_cols = [col for col in numerical_cols if col in df.columns]

        if numerical_cols:
            scaler = MinMaxScaler()
            numerical_data = pd.DataFrame(
                scaler.fit_transform(df[numerical_cols]),
                columns=numerical_cols,
                index=df.index
            )
        else:
            numerical_data = pd.DataFrame(index=df.index)

        # Weighted combination of features
        tfidf_weight = 0.1
        numerical_weight = 0.9

        tfidf_df *= tfidf_weight
        if not numerical_data.empty:
            numerical_data *= numerical_weight

        # Combine TF-IDF and numerical features
        feature_set = pd.concat([tfidf_df, numerical_data], axis=1)
        return feature_set

    def get_playlist_tracks(self, playlist_id):
        # Fetch tracks from a Spotify playlist
        playlist = []
        try:
            results = self.sp.playlist_tracks(playlist_id)
            while results:
                for item in results['items']:
                    track = item['track']
                    playlist.append({'SongName': track['name'], 'ArtistName': track['artists'][0]['name']})
                results = self.sp.next(results) if results['next'] else None
        except Exception as e:
            print("Error fetching playlist tracks:", e)
        return pd.DataFrame(playlist)

    def recommend_songs(self, playlist_id, top_n=10):
        # Get playlist tracks
        playlist_tracks = self.get_playlist_tracks(playlist_id)

        if playlist_tracks.empty:
            print("No tracks fetched from the playlist.")
            return pd.DataFrame()

        # Match tracks in the dataset
        matched_songs = self.spotify_df[
            self.spotify_df['SongName'].isin(playlist_tracks['SongName']) &
            self.spotify_df['ArtistName'].isin(playlist_tracks['ArtistName'])
        ]

        if matched_songs.empty:
            print("No matching tracks found in the dataset.")
            return pd.DataFrame()

        # Generate playlist vector
        playlist_vector = self.complete_feature_set.loc[matched_songs.index].mean(axis=0).values.reshape(1, -1)

        # Calculate similarity for non-playlist songs
        non_playlist_songs = self.complete_feature_set[~self.complete_feature_set.index.isin(matched_songs.index)].copy()
        non_playlist_songs['similarity'] = cosine_similarity(non_playlist_songs, playlist_vector)[:, 0]

        # Get top recommendations
        top_recommendations = non_playlist_songs.nlargest(top_n, 'similarity')
        recommendations = self.spotify_df.loc[top_recommendations.index, ['SongName', 'ArtistName']]

        # Ensure diversity in recommendations
        recommendations = recommendations.drop_duplicates(subset=['ArtistName'], keep='first')
        return recommendations.head(top_n)


print("model ready..")


model ready..


In [37]:
# Use the already authenticated sp object from your separate cell
model = RecommendationModel(sp)
playlist_id = '4GO0acBE8GssynB6dIIgrZ'  # Replace with a valid Spotify playlist ID
recommendations = model.recommend_songs(playlist_id, top_n=20)
print("Top 20 Recommendations:")
print(recommendations)

Spotify API client successfully initialized.
Top 20 Recommendations:
                                              SongName           ArtistName
847                         Tick Tock (feat. 24kGoldn)         Clean Bandit
468  The Rose Song - From "High School Musical: The...       Olivia Rodrigo
593                                   I Like Me Better  Anthony de la Torre
407                       Black Mask (feat. Pop Smoke)           Jay Gwuapo
271                         Another Life (feat. Alida)        Lucas & Steve
456                                             Bandey      The Local Train
433                                          Renegades        X Ambassadors
353                      Beside You (feat. Georgia Ku)               Matoma
905                                             Broken                 Lund
651                    STILL CHOSE YOU (feat. Mustard)        The Kid LAROI
13                                          goosebumps         Travis Scott
787                

### GUI with the link to spotify

In [43]:
import tkinter as tk
from tkinter import messagebox
import webbrowser  # To open Spotify links in the browser

class RecommendationUI:
    def __init__(self, root, recommendation_model):
        self.root = root
        self.root.title("Song Recommendation")
        
        # Create a frame for the input elements
        self.input_frame = tk.Frame(self.root)
        self.input_frame.pack(padx=10, pady=10)

        # Add a label for instructions
        self.welcome_label = tk.Label(
        self.input_frame,
        text=(
            "1. Go to your playlist in Spotify, look for its URL \n"
            "2. Example: For....' https://open.spotify.com/playlist/4GO0acBE8GssynB6dIIgrZ '....\n"
            "3. So your playlist ID is.....'4GO0acBE8GssynB6dIIgrZ'....."
        ),
        font=("Helvetica", 10),
        
        justify="left"
        )
        self.welcome_label.grid(row=0, column=0, columnspan=3, pady=10)

        # Create input label and entry for playlist ID
        self.playlist_label = tk.Label(self.input_frame, text="Enter Playlist ID:")
        self.playlist_label.grid(row=1, column=0, padx=10, pady=10)

        self.playlist_entry = tk.Entry(self.input_frame, width=40)
        self.playlist_entry.grid(row=1, column=1, padx=10, pady=10)

        # Create button to trigger recommendation
        self.recommend_button = tk.Button(self.input_frame, text="Recommend", command=self.get_recommendations)
        self.recommend_button.grid(row=1, column=2, columnspan=2, pady=10)

        # Create a frame for the recommendations
        self.recommendations_frame = tk.Frame(self.root)
        self.recommendations_frame.pack(padx=10, pady=10)

        # Store the recommendation model
        self.recommendation_model = recommendation_model

    def get_recommendations(self):
        """Get recommendations based on the playlist ID and display them as buttons."""
        playlist_id = self.playlist_entry.get().strip()
        
        if not playlist_id:
            messagebox.showerror("Input Error", "Please enter a valid Playlist ID, I wont be able to give my recommendation")
            return

        try:
            recommendations = self.recommendation_model.recommend_songs(playlist_id)
            if recommendations.empty:
                messagebox.showinfo("No Recommendations", "No recommendations found for the given playlist.")
            else:
                # Clear previous buttons
                for widget in self.recommendations_frame.winfo_children():
                    widget.destroy()

                # Create a button for each recommendation
                for index, row in recommendations.iterrows():
                    song_text = f"{row['SongName']} - {row['ArtistName']}"
                    spotify_link = self.get_spotify_link(row)
                    # Create a button that opens the Spotify link when clicked
                    recommendation_button = tk.Button(self.recommendations_frame,width=65, text=song_text, command=lambda link=spotify_link: self.open_spotify(link))
                    recommendation_button.pack(fill=tk.X, padx=5, pady=5)

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred while fetching recommendations: {str(e)}")

    def get_spotify_link(self, row):
        """Generate the Spotify link for a song."""
        song_name = row['SongName'].replace(" ", "%20")  # Replace spaces for URL formatting
        artist_name = row['ArtistName'].replace(" ", "%20")
        return f"https://open.spotify.com/search/{song_name}%20{artist_name}"

    def open_spotify(self, link):
        """Open the provided Spotify link in the default web browser."""
        webbrowser.open(link)

if __name__ == "__main__":
    root = tk.Tk()
    # Initialize RecommendationModel with Spotify client
    recommendation_model = RecommendationModel(sp)
    ui = RecommendationUI(root, recommendation_model)
    root.mainloop()


Spotify API client successfully initialized.
